# Задача: предсказание победы по данным о первых 5 минутах игры

По первым 5 минутам игры предсказать, какая из команд победит: Radiant или Dire?

In [5]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import numpy as np

import warnings
warnings.filterwarnings('ignore')

## Подход 1: градиентный бустинг "в лоб"

Считаем таблицу с признаками из файла features.csv. Удалим признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке). 

In [6]:
import pandas as pd
features = pd.read_csv('features.csv', index_col='match_id')
y_train = features.radiant_win


features = features.drop(columns = ['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'])
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


Проверим выборку на наличие пропусков с помощью функции count().

In [7]:
val_nan = features.isna().sum() # посчитаем количество Nan в каждом столбце

names_nan = {}
for i in range(len(val_nan)):   #переведем наши значения в словарь для удобства
    names_nan[features.columns[i]] = val_nan[i]

name = names_nan.copy()         #удалим те столбцы, где нет Nan
for key, value in name.items():
    if value == 0:
        del names_nan[key]
names_nan 

{'first_blood_time': 19553,
 'first_blood_team': 19553,
 'first_blood_player1': 19553,
 'first_blood_player2': 43987,
 'radiant_bottle_time': 15691,
 'radiant_courier_time': 692,
 'radiant_flying_courier_time': 27479,
 'radiant_first_ward_time': 1836,
 'dire_bottle_time': 16143,
 'dire_courier_time': 676,
 'dire_flying_courier_time': 26098,
 'dire_first_ward_time': 1826}

#### Вопрос №1: Какие признаки имеют пропуски среди своих значений (приведите полный список имен этих признаков)? Что могут означать пропуски в этих признаках?

$\textbf{Ответ:}$ 
Полный список признаков, имеющих пропущенные значения:

`first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time 
dire_flying_courier_time
dire_first_ward_time`

Например, `first_blood_time` и `first_blood_team`. Пропуски в данных признаках могут означать, что команда, совершившая первую кровь, а значит, и игровое время первой крови не произошли за первые 5 минут игры.

Заменим пропуски на нули с помощью функции fillna().

In [8]:
features = features.fillna(0)

Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируем генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудем перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оценим качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуем при этом разное количество деревьев (10, 20, 30).

In [9]:
from sklearn import model_selection
from sklearn.ensemble import GradientBoostingClassifier
import time
import datetime

In [10]:
kf = model_selection.KFold(n_splits = 5, shuffle = True)
trees = [30]#10, 20]

start_time = datetime.datetime.now()

for tree in trees:
    clf = GradientBoostingClassifier(n_estimators = tree)
    clf_fit = clf.fit(features, y_train)
    scores = model_selection.cross_val_score(clf_fit, features, y_train, cv = kf, scoring = 'roc_auc')
    score_mean = scores.mean()
    time = datetime.datetime.now() - start_time
    print('Количество деревьев: {}, качество: {}, затраченное время: {}'.format(tree, score_mean, time))
  

Количество деревьев: 30, качество: 0.689905636334161, затраченное время: 0:03:05.591398


#### Вопрос №3: Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями?
$\textbf{Ответ: }$ Качество, полученное при 30 деревьях = 0.689,а затраченное время при этом = 7 минут. 

#### Вопрос №4: Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что можно сделать, чтобы ускорить его обучение при увеличении количества деревьев?

$\textbf{Ответ: }$ При увеличении количества деревьев, качество упало. Для того, чтобы ускорить обучение при увеличении числа деревьев, можно использовать для обучения и кросс-валидации не всю выборку, а, например, половину объектов или же уменьшить глубину деревьев.

## 2. Подход 2: логистическая регрессия

Оценим качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберем при этом лучший параметр регуляризации (C).

In [11]:
scaler = StandardScaler()
scaler.fit(features)
features_scal = scaler.transform(features)

In [12]:
kf = model_selection.KFold(n_splits = 5, shuffle = True)
regressor = LogisticRegression(penalty = 'l2')

start_time = datetime.datetime.now()

log_reg = model_selection.GridSearchCV(regressor, param_grid =  {'C': np.power(10.0, np.arange(-5, 6))}, scoring='roc_auc', cv = kf)
log_reg_fit = log_reg.fit(features_scal, y_train)  

time = datetime.datetime.now() - start_time
print(time)

0:00:50.644725


In [13]:
log_reg_fit.best_score_

0.7163647186241546

In [14]:
log_reg_fit.best_params_ 

{'C': 0.01}

#### Вопрос №1: Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем можно объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

$\textbf{Ответ: }$ Качество у логистической регрессии над всеми исходными признаками получилось: 0.716395 по сравнению с градиентным бустингом качество увеличилось примерно на 0,03. Работает быстрее в 7 раз. В данном случае бустинг состоит из 30 слабых моделей, в то время как логистическая регрессия - это 1 модель.

Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации.

In [15]:
features_w_cat  = features.drop(columns = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero'
                   , 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'])

In [16]:
scaler = StandardScaler()
scaler.fit(features_w_cat)
features_scal_cat = scaler.transform(features_w_cat)

In [17]:
log_reg = model_selection.GridSearchCV(regressor, param_grid =  {'C': np.power(10.0, np.arange(-5, 6))}, scoring='roc_auc', cv = kf)
log_reg_fit_cat = log_reg.fit(features_scal_cat, y_train)

In [18]:
log_reg_fit_cat.best_score_

0.7164521898594021

In [19]:
log_reg_fit_cat.best_params_

{'C': 0.01}

#### Вопрос № 2: Как влияет на качество логистической регрессии удаление категориальных признаков? Чем можно объяснить это изменение?

$\textbf{Ответ: }$ Качество без категориальных признаков = 0,716346. Упало на 0.00005, незначительно, что говорит о том, что категориальные признаки не имели особого веса.

На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясним из данных, сколько различных идентификаторов героев существует в данной игре.

In [20]:
feat = pd.read_csv('features.csv', index_col='match_id')
hero_col = ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero','d1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
    
unique_hero = set()
    
for row in hero_col:
    for id in feat[row].unique():
        unique_hero.add(id)

print(len(unique_hero))

108


#### Вопрос №3: Сколько различных идентификаторов героев существует в данной игре?

$\textbf{Ответ: }$ 108

Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire.

In [21]:
X_pick_u = np.zeros((feat.shape[0], 113))
for i, match_id in enumerate(feat.index):
    for p in range(5):
        X_pick_u[i, feat.loc[match_id, 'r%d_hero' % (p+1)] -1] = 1
        X_pick_u[i, feat.loc[match_id, 'd%d_hero' % (p+1)] -1] = -1

In [22]:
features_bag = np.concatenate([features_scal_cat, X_pick_u], axis = 1)

Проведем кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации.

In [23]:
log_reg = model_selection.GridSearchCV(regressor, param_grid =  {'C': np.power(10.0, np.arange(-5, 6))}, scoring='roc_auc', cv = kf)
log_reg_fit_bag = log_reg.fit(features_bag, y_train)

In [24]:
log_reg_fit_bag.best_score_

0.7518168833754922

#### Вопрос №4: Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем можно это объяснить?

$\textbf{Ответ: }$Качество после добавления мешка слов: 0.752. Улучшилось на 0.04. Преобразовали признаки, которые влияют на исход игры.

Построим предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедимся, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [25]:
features_test = pd.read_csv('./features_test.csv', index_col='match_id')
features_test = features_test.fillna(0)

In [26]:
features_test_cat = features_test.drop(columns = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
                        'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'])

In [27]:
df = pd.read_csv('features_test.csv', index_col='match_id')
hero_cols = ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero','d1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
    
unique_heroes_list = set()
    
for row in hero_cols:
    for id in df[row].unique():
        unique_heroes_list.add(id)
        
X_pick = np.zeros((df.shape[0], 113))
for i, match_id in enumerate(df.index):
    for p in range(5):
        X_pick[i, df.loc[match_id, 'r%d_hero' % (p+1)] -1] = 1
        X_pick[i, df.loc[match_id, 'd%d_hero' % (p+1)] -1] = -1

In [28]:
scaler = StandardScaler()
scaler.fit(features_test_cat)
features_test_scal_cat = scaler.transform(features_test_cat)

In [29]:
features_test_bag = np.concatenate([features_test_scal_cat, X_pick], axis = 1)

In [30]:
y_pred = log_reg.predict_proba(features_test_bag)[:, 1]

print('Минимальное значение прогноза: ', min(y_pred))
print('Максимальное значение прогноза: ', max(y_pred))

Минимальное значение прогноза:  0.008583784559517813
Максимальное значение прогноза:  0.9964598059914422


#### Вопрос № 5: Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

$\textbf{Ответ: }$ Минимальное значение прогноза:  0.008583784554994622 \\
Максимальное значение прогноза:  0.9964598059895592

In [31]:
predictions = pd.DataFrame({'match_id':[i for i in features_test.index], 'radiant_win': y_pred})

#predictions.to_csv('result.csv', sep = ',', encoding = 'utf-8', index = False)
#### score на Kaggle 0.75526 